<a href="https://colab.research.google.com/github/Anshuman-02905/Custom_Satellite_Scripts/blob/main/getImages_and_Masks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install geemap
!pip install geopandas
!apt-get update
!apt-get install libgdal-dev -y
!apt-get install python-gdal -y
!apt-get install python-numpy python-scipy -y

In [2]:
import geemap
import os
import glob
import gdal ## fingers crossed!
import os
import ee
import pandas as pd
import numpy as np
import math
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, Polygon
import folium
import os

In [3]:
os.mkdir("Sentinel_png")
os.mkdir("Mask_png")
os.mkdir("Sentinel_tif")
os.mkdir("Urban_tif")


In [14]:
Map=geemap.Map() 

In [21]:
Map=geemap.Map()
#state = ee.FeatureCollection("FAO/GAUL/2015/level2").filter(ee.Filter.eq('ADM2_NAME','Bangalore Urban'))
sentinel = ee.ImageCollection('COPERNICUS/S2_SR').select(['B4', 'B3', 'B2']) \
                  .filterDate('2017-01-01', '2020-01-30') \
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',5)) \
                  .filterBounds(state)
image = sentinel.mean().divide(10000).clip(state).select(['B4', 'B3', 'B2']).unmask()

visualization = {
  'min': 0.0,
  'max': 0.3,
  'bands': ['B4', 'B3', 'B2'],
  }
#Map.addLayer(sentinel.mean().divide(10000).clip(state), visualization, 'RGB')

mask = ee.Image("DLR/WSF/WSF2015/v1").clip(state)

visualization = {
  'min': 0,
  'max': 255,
}

#Map.addLayer(urban.clip(state), visualization, "Human1")
mask=mask.expression('b("settlement") == 255 ? 255:0',{'settlement':mask.select('settlement')}).clip(state)

Map.addLayer(mask, visualization, "Human2")
Map.centerObject(state)
Sentinel="/content/Sentinel_tif/"+"sentinel.tif"
urban="/content/Urban_tif/"+"final.tif"

geemap.ee_export_image(mask, filename=urban, scale=50, region=state.geometry(), file_per_band=False)
geemap.ee_export_image(image, filename=Sentinel, scale=50, region=state.geometry(), file_per_band=False)
text1="/content/Sentinel_png/"+"sentinel"+".png"
text2="/content/Mask_png/"+"final"+".png"
options_list = ['-ot Byte','-of PNG','-scale 0 0.3 0 255'] 
options_string = " ".join(options_list) 
gdal.Translate(text1,Sentinel, options=options_string)
options_list = ['-ot Byte','-of PNG'] 
options_string = " ".join(options_list)
gdal.Translate(text2, urban, options=options_string) 




Generating URL ...
Please wait ...
Data downloaded to /content/Urban_tif/final.tif
Generating URL ...
Please wait ...
Data downloaded to /content/Sentinel_tif/sentinel.tif


<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7efc3173bc90> >

In [12]:
Map

Map(center=[12.935767413564722, 77.57391086572378], controls=(WidgetControl(options=['position', 'transparent_…

In [22]:
state = ee.FeatureCollection("FAO/GAUL/2015/level2").filter(ee.Filter.eq('ADM2_NAME','Bangalore Urban'))
shps=state.getInfo()['features'][0]['geometry']['coordinates']

In [23]:
#state.getInfo()['features'][0]['geometry']['coordinates']

In [24]:
final_list = [item for sublist in shps for item in sublist]
min_lat=1000;
min_lon=1000;
max_lat=-1000;
max_lon=-1000;
for i in final_list:
  min_lat=min(min_lat,i[0])
  min_lon=min(min_lon,i[1])
  max_lat=max(max_lat,i[0])
  max_lon=max(max_lon,i[1])
min_lon-=0.00098
max_lon+=0.00998
left_upper=tuple([min_lat,min_lon])
left_lower=tuple([max_lat,min_lon])
right_lower=tuple([max_lat,max_lon])
right_upper=tuple([min_lat,max_lon])

In [25]:
def get_distance(LA1,LN1,LA2,LN2):
  return math.sqrt(math.pow((LA1-LA2),2)+math.pow((LN1-LN2),2))

In [26]:
left_dist=get_distance(left_upper[0],left_upper[1],left_lower[0],left_lower[1])
print(left_dist)
right_dist=get_distance(right_upper[0],right_upper[1],right_lower[0],right_lower[1])
print(right_dist)
upper_dist=get_distance(left_upper[0],left_upper[1],right_upper[0],right_upper[1])
print(upper_dist)
lower_dist=get_distance(left_lower[0],left_lower[1],right_lower[0],right_lower[1])
print(lower_dist)

0.5033789690619983
0.5033789690619983
0.5859878671830234
0.5859878671830234


In [ ]:
final_list=list()
Lat=12.971599
Lon=77.594566
c=0
for i in range(0,10):
  Lon=0
  for j in range(0,10):
    LL=[Lat,Lon]
    UL=[Lat+left_dist,Lon]
    LR=[Lat,Lon+lower_dist]
    UR=[Lat+left_dist,Lon+lower_dist]
    final_list=[LL,LR,UR,UL,LL]
    result=get_boundaries(final_list,c)
    if result!=-1:
      c+=1
    #print(final_list)
    Lon+=lower_dist
  Lat+=left_dist

/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  


Generating URL ...
Please wait ...
Data downloaded to /content/Urban_tif/0.tif
Generating URL ...
Please wait ...
Data downloaded to /content/Sentinel_tif/0.tif


/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  


Generating URL ...
Please wait ...
Data downloaded to /content/Urban_tif/1.tif
Generating URL ...
An error occurred while downloading.
Image.divide: If one image has no bands, the other must also have no bands. Got 0 and 1.
Exception


/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  


...

Generating URL ...
Please wait ...
Data downloaded to /content/Urban_tif/56.tif
Generating URL ...
Please wait ...
Data downloaded to /content/Sentinel_tif/56.tif


/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  


Generating URL ...
Please wait ...
Data downloaded to /content/Urban_tif/57.tif
Generating URL ...
Please wait ...
Data downloaded to /content/Sentinel_tif/57.tif


/usr/local/lib/python3.7/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  


Generating URL ...
Please wait ...


In [26]:
#final_list=[left_upper,left_lower,right_lower,right_upper,left_upper]

In [29]:
def get_boundaries(final_list,c):
  coordinates = final_list
  def create_polygon(coords, polygon_name):
    ''' Create a polygon from coordinates'''
    polygon = Polygon(coordinates)
    gdf = gpd.GeoDataFrame(crs = {'init' :'epsg:4326'})
    gdf.loc[0,'name'] = polygon_name
    gdf.loc[0, 'geometry'] = polygon
    return gdf
  shapefile = create_polygon(coordinates, "Amesterdam")
  shapefile.to_file("myboundary.shp")
  mine_shp="/content/myboundary.shp"
  mine = geemap.shp_to_ee(mine_shp) 
  return func(c)
  

In [28]:
def func(c):
  try:
    mine_shp="/content/myboundary.shp"
    mine = geemap.shp_to_ee(mine_shp) 
    Map=geemap.Map()
    state = mine
    Map=geemap.Map()
    #state = ee.FeatureCollection("FAO/GAUL/2015/level2").filter(ee.Filter.eq('ADM2_NAME','Bangalore Urban'))
    sentinel = ee.ImageCollection('COPERNICUS/S2_SR').select(['B4', 'B3', 'B2']) \
                  .filterDate('2017-01-01', '2020-01-30') \
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',5)) \
                  .filterBounds(state)
    image = sentinel.mean().divide(10000).clip(state).select(['B4', 'B3', 'B2']).unmask()

    visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
    }
    #Map.addLayer(sentinel.mean().divide(10000).clip(state), visualization, 'RGB')

    mask = ee.Image("DLR/WSF/WSF2015/v1").clip(state)

    visualization = {
    'min': 0,
    'max': 255,
    }

    #Map.addLayer(urban.clip(state), visualization, "Human1")
    mask=mask.expression('b("settlement") == 255 ? 255:0',{'settlement':mask.select('settlement')}).clip(state)

    #Map.addLayer(mask, visualization, "Human2")
    #Map.centerObject(state)
    Sentinel="/content/Sentinel_tif/"+str(c)+".tif"
    urban="/content/Urban_tif/"+str(c)+".tif"

    geemap.ee_export_image(mask, filename=urban, scale=50, region=state.geometry(), file_per_band=False)
    geemap.ee_export_image(image, filename=Sentinel, scale=50, region=state.geometry(), file_per_band=False)
    text1="/content/Sentinel_png/"+str(c)+".png"
    text2="/content/Mask_png/"+str(c)+".png"
    options_list = ['-ot Byte','-of PNG','-scale 0 0.3 0 255'] 
    options_string = " ".join(options_list) 
    gdal.Translate(text1,Sentinel, options=options_string)
    options_list = ['-ot Byte','-of PNG'] 
    options_string = " ".join(options_list)
    gdal.Translate(text2, urban, options=options_string) 

  except:
    print("Exception")
    return -1
